# Machine Learning @ UWr 2020

**Lecture 03 part 2**

## Naive Bayes classifier

We will now develop a classifier for categorical data which follows straigth from the Bayes theorem.

The Naive Bayes is surprisingly strong on text data, hance we will introduce it in this context.

Suppose you want to classify a document as SPAM/NONSPAM (HAM) we want

\begin{equation}
    p(\text{S}|\text{T})
\end{equation}

where $\text{S}$ is a random variable taking values $s$[pam] and $h$[am] and $\text{T}$ is a random variable representing the text to classify.

using the Bayes theorem we get

\begin{equation}
    p(\text{S}|\text{T}) =
    \frac{p(\text{T}|\text{S})p(\text{S})}{p(\text{T})}
\end{equation}

The Bayes theorem allows us to express a classification problem as a generation problem: we will create a model $p(\text{T}|\text{S})$ for generating texts and combine it with the prior probability of getting a spam $p(\text{S})$. However, we will not need  $p(\text{T})$: the probability of ever seeing a given document.

To estimate $p(\text{T}|\text{S})$ we need to define a data generation model. A text is a sequence of words:
$$
\text{T} = W_1, W_2, W_3,\ldots,W_n.
$$
Thus, 
$$
p(\text{T}|\text{S}) = p(W_1|\text{S})p(W_2|W_1,\text{S})p(W_n|W_1, ..., W_{n-1}, \text{S})
$$
We will further simplify this by (naively) assuming that 

\begin{equation}
    \begin{split}
 p(\text{T}|\text{S}) &=  (W_1|\text{S})p(W_2|W_1,\text{S})p(W_n|W_1, ..., W_{n-1}, \text{S}) \\
 &\approx p(W_1|\text{S})p(W_2|\text{S})p(W_n|\text{S}) \\
 &= \prod_{W_i \in \text{T}}p(W_i|\text{S})
\end{split}
\end{equation}

This corresponds to a generative model in which the sender first flips a biased coin to see if the generated document will be a spam or a ham one. Then, he picks a box labeled spam or ham. Finally, the sender draws with replacement words from the appropriate box.

The full sampling model has the following parameters:
1. $\phi$ - the probability of generating a Spam.
2. $\theta_{w,s}$ - the probability of generating word $w$ in a S document, $\sum_w \theta_{w,s}=1$,pam
3. $\theta_{w,h}$ - the probability of generating word $w$ in a Ham document, $\sum_w \theta_{w,h}=1$.

All parameters are easy to estimate using maximum likelihood principle:
1. $\phi = p(\text{S}=s)$ is just the fraction of all spams in our corpus.
2. $\theta_{w,s} = p(W=w|S=s)$ is the fraction of the number of occurrences of word $w$ in all spams.
3. $\theta_{w,h} = p(W=w|S=h)$ is the fraction of the number of occurrences of word $w$ in all non-spams.

The derivation of the MLE result above is pretty technical and similar to the MLE estimator for Bernulli variables from Homework 1. The ony change is that handling the constraints on $\Theta_{w,s}$ and $\Theta_{w,h}$ requires using Langrange multipliers.


### Example:

suppose our corpus has 4 documents:
1. "buy much now": Spam
2. "many dollars gain": Spam
3. "like you much": Ham
4. "do your nice homework": Ham

Then:
$\phi = p(\text{S}=s) = 2/4 = 0.5$

$\theta_{w,h}$ is given by the following table

|       | buy | much | now | dollars | gain | like | you/your | do  | homework | nice |
|------|-----|------|-----|---------|------|------|----------|-----|----------|------|
| Spam | 1/6 | 2/6  | 1/6 | 1/6     | 1/6  | 0/6  | 0/6      | 0/6 | 0/6      | 0/6  |
| Ham  | 0/7 | 1/7  | 0/7 | 0/7     | 0/7  | 1/7  | 2/7      | 1/7 | 1/7      | 1/7  |

To classify a new phrase "much much gain" we compute

$$
\begin{split}
&p(\text{S} = s | \text{"much much"}) \\
&= p(\text{S}=s) p(\text{much}|\text{S}=s)p(\text{much}|\text{S}=s) / p(\text{T} = \text{"much much"}) \\
&= 1/2 \cdot 2/6 \cdot 2/6 \cdot 1/Z = 4/36 \cdot 1/Z
\end{split}
$$

$$
\begin{split}
&p(\text{S} = h | \text{"much much"}) \\
&= p(\text{S}=h) p(\text{much}|\text{S}=h)p(\text{much}|\text{S}=h) / p(\text{T} = \text{"much much"}) = \\
&= 1/2 \cdot 1/7 \cdot 1/7  \cdot 1/Z = 1/49 \cdot 1/Z
\end{split}
$$

We now solve for $Z$, knowing that $p(\text{S} = s | \text{"much much"}) + p(\text{S} = h | \text{"much much"}) = 1$:

$$
\frac{4}{36Z} + \frac{1}{49Z} = 1 \rightarrow Z\approx0.13
$$

Finally, we recover the proabilities:
$$
\begin{split}
p(\text{S} = s | \text{"much much"}) &= 4/36 \cdot 1/Z \approx 0.84 \\
p(\text{S} = h | \text{"much much"}) &= 1/49 \cdot 1/Z \approx 0.16
\end{split}
$$

Thus the text is classified as Spam.

Lets now see what happens if the text contains missing word, e.g. "do gain much" (written in the perfect ex-military crown prince English grammar):

$$
\begin{split}
&p(\text{S} = s | \text{"do gain much"}) \\
&= p(\text{S}=s) p(\text{do}|\text{S}=s)p(\text{gain}|\text{S}=s)p(\text{much}|\text{S}=s) / p(\text{T} = \text{"do gain much"}) = \\
&= 1/2 \cdot 0/6 \cdot 1/6 \cdot 2/6 \cdot 1/Z = 0
\end{split}
$$

$$
\begin{split}
&p(\text{S} = h | \text{"do gain much"}) \\
&= p(\text{S}=h) p(\text{do}|\text{S}=h)p(\text{gain}|\text{S}=h)p(\text{much}|\text{S}=h) / p(\text{T} = \text{"do gain much"}) = \\
&= 1/2 \cdot 1/7 \cdot 0/7 \cdot 1/7 \cdot 1/Z = 0
\end{split}
$$

Now the model is unable to make a decision!


The problem stems from the fact, that we are modeling rare events with MLE and just like in the polling example, estimating frequecies with MLE doesn't make sense whane we have little data. 

Inspired by the Bayesian approach to polling (which realli is about estimating counts) a common technique, called Laplace smoothing, is to assume that each word in the vocabulary was seen a given number of times in each kind of document. These fictional seightings of words are often called "pseudocounts".
 
With Laplace smoothing (assuming each word occurred 0.5 times in a virtual spam document and 0.5 times in a virtual ham one) the table becomes
 
|       | buy | much | now | dollars | gain | like | you/your | do  | homework | nice |
|------|-----|------|-----|---------|------|------|----------|-----|----------|------|
| Spam | 1.5/11 | 2.5/11  | 1.5/11 | 1.5/11     | 1.5/11  | 0.5/11  | 0.5/11      | 0.5/11 | 0.5/11      | 0.5/11  |
| Ham  | 0.5/12 | 1.5/12  | 0.5/12 | 0.5/12     | 0.5/12  | 1.5/12  | 2.5/12      | 1.5/12 | 1.5/12      | 1.5/12  |

Now:
$$
\begin{split}
&p(\text{S} = s | \text{"do gain much"}) \\
&= p(\text{S}=s) p(\text{do}|\text{S}=s)p(\text{gain}|\text{S}=s)p(\text{much}|\text{S}=s) / p(\text{T} = \text{"do gain much"}) = \\
&= 1/2 \cdot 0.5/11 \cdot 1.5/11 \cdot 2.5/11 \cdot 1/Z = 1.875/2662 \cdot 1/Z
\end{split}
$$

$$
\begin{split}
&p(\text{S} = h | \text{"do gain much"}) \\
&= p(\text{S}=h) p(\text{do}|\text{S}=h)p(\text{gain}|\text{S}=h)p(\text{much}|\text{S}=h) / p(\text{T} = \text{"do gain much"}) = \\
&= 1/2 \cdot 1.5/12 \cdot 0.5/12 \cdot 1.5/12 \cdot 1/Z = 1.125 / 3456 \cdot 1/Z
\end{split}
$$

Again, we can work out the value of $Z$ and obtain the final probabilities:

$$
\begin{split}
p(\text{SPAM} = s | \text{"do much gain"}) &= 68.4\%\\
p(\text{SPAM} = h | \text{"do much gain"}) &= 31.6\%
\end{split}
$$

Thus the model predicts SPAM with a fairly large confidence.